# Laboratório 2 – Aprendizado de Máquina Probabilístico

**Instituto Tecnológico de Aeronáutica – ITA**

**Inteligência Artificial – CMC-15**

**Professores:** 

Paulo André Castro

Paulo Marcelo Tasinaffo

**Alunos:**

Álvaro Tedeschi Neto

Davi Muniz de Vasconcelos

Gabriel Henrique Gobi

Thiago Lopes de Araujo

---

In [1]:
# for random decision of the tree criterias'
import random

# for calculating entropy
import math

# for data manipulation
import pandas as pd

# for train and test split
from sklearn.model_selection import train_test_split

## 2.1 Dataset

In `republican_democrat.txt` we have the following description of the data:

```
This data set includes votes for each of the U.S. House of Representatives Congressmen on the 16 key votes identified by the CQA.  The CQA lists nine different types of votes: voted for, paired for, and announced for (these three simplified to yea), voted against, paired against, and announced against (these three simplified to nay), voted present, voted present to avoid conflict of interest, and did not vote or otherwise make a position known (these three simplified to an unknown disposition).
``` 

Let's see how the data looks:

In [2]:
df = pd.read_csv("republican_democrat.csv")
print(f'The dataset has {df.shape[0]} entries')
df.head()

The dataset has 415 entries


,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,religious-groups-in-schools,anti-satellite-test-ban,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,Target
0,n,y,n,y,y,n,n,y,?,y,y,y,n,y,republican
1,n,y,n,y,y,n,n,n,n,y,y,y,n,?,republican
2,?,y,y,?,y,n,n,n,y,n,y,y,n,n,democrat
3,n,y,y,n,y,n,n,n,y,n,y,n,n,y,democrat
4,y,y,y,n,y,n,n,n,y,?,y,y,y,y,democrat


All features are ***categoric features***. The columns above represent, accordingly with the text document provided:

```
1. Class Name: 2 (democrat, republican)
2. handicapped-infants: 2 (y,n)
3. water-project-cost-sharing: 2 (y,n)
4. adoption-of-the-budget-resolution: 2 (y,n)
5. physician-fee-freeze: 2 (y,n)
6. religious-groups-in-schools: 2 (y,n)
7. anti-satellite-test-ban: 2 (y,n)
8. mx-missile: 2 (y,n)
9. immigration: 2 (y,n)
10. synfuels-corporation-cutback: 2 (y,n)
11. education-spending: 2 (y,n)
12. superfund-right-to-sue: 2 (y,n)
13. crime: 2 (y,n)
14. duty-free-exports: 2 (y,n)
15. export-administration-act-south-africa: 2 (y,n)
```

## 2.2 Tasks

### 2.2.1 Decision Tree Classifier

For "missing" values (?), we will not do any pre-processing such as data removal or inputation. In this case, the information of the "missing" data is relevant, since it can show a neutrality of opinion on the part of the party's candidates with respect to the themes asked to then.

Splitting the dataset on train and test sets:

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['Target'], axis=1), 
    df['Target'],  
    test_size=0.2,  
    random_state=0)  

X_train.shape, X_test.shape

((332, 14), (83, 14))

The next classes will implement a tree data structure and the Decision Tree classifier that will be used for this problem.

#### TreeNode, DecisionNode and TargetNode

In [4]:
class TreeNode:
    """
    Auxiliary class for the nodes in the Decision Tree.
    """ 
    def __init__(self):
        """
        Creates a generic node.
        This class will not be instatiated.
        """
        self.parent = None
        self.value = None
        self.children = None
    
    def add_parent(self, node):
        """
        Adds a node parent to this node.

        :arg node: The parent node.
        :type node: TreeNode.
        """
        self.parent = node
        
    def is_leaf(self):
        """
        Checks if this node is a leaf or not.
        
        :return: If the node is a leaf.
        :return type: Bool.
        """
        raise NotImplementedError("This is an abstract class.")
                   
class DecisionNode(TreeNode):
    """
    Type of the inner nodes of the Decision Tree.
    This type of node has a criteria and children.
    The children are linked by a value of the criteria.
    """
    def __init__(self, criteria, default):
        """
        Creates a decision node with a criteria.
        
        :arg criteria: The decision criteria for this node.
        :type criteria: String.
        :arg default: The default target value for this node, when it has no decision for an asked value.
        :type default: Object.
        """
        super().__init__()
        self.value = criteria
        self.default = default
        self.children = {}
        
    def add_children(self, value, node):
        """
        Add a child to the node for criteria equal value.
        
        :arg value: Value of the edge which the child is linked by criteria.
        :type value: Object.
        :arg node: Child node to be linked to this node.
        :type node: TreeNode.
        """
        node.add_parent(self)
        self.children[value] = node
    
    def get_criteria(self):
        """
        Gets the criteria used for decision.
        
        :return: Criteria value.
        :return type: String.
        """
        return self.value
        
    def get_child(self, value):
        """
        Gets the child of this node that is linked by a edge with some value for criteria.
        
        :arg value: Value of the edge which the child is linked by criteria. 
        :return: The child of the node which edge has value equals criteria.
        :return type: TreeNode.
        
        """
        if not value in self.children.keys():
            self.children[value] = TargetNode(self.default)
        return self.children[value]
    
    
    def is_leaf(self):
        """
        Checks if this node is a leaf or not.
        
        :return: If the node is a leaf.
        :return type: Bool.
        """
        return False

class TargetNode(TreeNode):
    """
    Type of the leaf nodes of the Decision Tree.
    This type of node has an value for the decision target and no children.
    """
    def __init__(self, target_value):
        """
        Creates a target node with a target value for decision.
        
        :arg target_value: The value to be predicted on decision at this node.
        :type target_value: Object.
        """
        super().__init__()
        self.value = target_value
    
    def get_target_value(self):
        """
        Gets the target value for predicting at this node.
        
        :return: The target value for decision.
        :return type: Object.
        """
        return self.value
    
    def is_leaf(self):
        """
        Checks if this node is a leaf or not.
        
        :return: If the node is a leaf.
        :return type: Bool.
        """
        return True
        

#### DecisionTreeClassifier

In [5]:
class DecisionTreeClassifier:
    """
    Class for the ML model for supervised learning to make classification based on Decision Tree.
    """   
    def __init__(self, splitter="best", max_depth=None):
        """
        Creates an instance of the classifier.
        """
        if splitter != "best" and splitter != "random":
            raise Exception("Invalid splitter method. Accepted values are [\"best\", \"random\"].")
        self._splitter = splitter
        self._max_depth = max_depth
        self.decision_tree = None
    
    def fit(self, X, y):
        """
        Uses train data to generate the decision tree model used for prediction.
        
        :arg X: Data used for train the model with all features that can be used by the model.
        :type X: pandas DataFrame.
        :arg y: Data column with the target values for the supervised learning. Should be a binary target.
        :type y: pandas Series.
        """
        self.decision_tree = self._decision_tree_learn(X, y, X.columns.values.tolist(), 0)
        
    def predict(self, X):
        """
        Makes predictions for the target value given a test dataset.
        
        :arg X: Data to be used as an input to the model to make class predicition.
        :type X: pandas DataFrame.
        :return: Predictions made by the model.
        :return type: pandas Series.
        """
        if self.decision_tree is None:
            raise Exception("The model was not trained yet. Run fit method first.")
        y_predicted = []
        for index, row in X.iterrows():
            node = self.decision_tree # root node
            while not node.is_leaf():
                criteria = node.get_criteria()
                node = node.get_child(row[criteria])
            y_predicted.append(node.get_target_value())
        return pd.Series(y_predicted)
            
    def _decision_tree_learn(self, X, y, features, depth):
        """
        Recursive method to generate and update the decision tree model on training.
        
        :arg X: Subset of train data to be used to create a new subtree of the model.
        :type X: pandas Dataframe.
        :arg y: Subset of target values to be used to create a new subtree of the model.
        :type y: pandas Series.
        :arg features: List of features names used to create the criterias of this subtree of the model. 
        :type features: List of strings.
        :arg depth: Depth of the current decision tree.
        :type depth: Int.
        :return: Subtree of the decision tree model.
        :return type: TreeNode.
        """
        if X_train.shape[0] == 0:
            return None
        
        elif len(y.unique()) == 1:
            return TargetNode(y.unique()[0])
        
        elif len(features) == 0 or depth == self._max_depth - 1:
            return TargetNode(y.mode()[0]) 
                
        else:
            if self._splitter == "random":
                criteria = random.choice(features)
            elif self._splitter == "best":
                criteria = self._get_best_feature(X, y, features)
                
            m = y.mode()[0]
            tree = DecisionNode(criteria, m)   
            for value in X[criteria].unique():
                X_subset = X[X[criteria]==value]
                y_subset = y[X[criteria]==value]
                features_subset = list(features)
                features_subset.remove(criteria)
                subtree = self._decision_tree_learn(X_subset, y_subset, features_subset, depth+1)
                tree.add_children(value, subtree)
            return tree
        
    def _get_best_feature(self, X, y, features):
        """
        Gets the best feature to use as a criteria on a decision node.
        
        :arg X: Subset of train data to be used to choose the best criteria.
        :type X: pandas Dataframe.
        :arg y: Subset of target values to be used to choose the best criteria.
        :type y: pandas Series.
        :arg features: List of features names used to choose the best criteria . 
        :type features: List of strings.
        :return: the best criteria of the features that maximizes information gain.
        :return type: String.
        """
        best = ''
        best_gain = -math.inf
        entropy = self._get_entropy(y)
        for feature in features:
            new_entropy = 0
            for value in X[feature].unique():
                subset_y = y[X[feature]==value]
                new_entropy += subset_y.count()/y.count() * self._get_entropy(subset_y)
            info_gain = entropy - new_entropy
            if info_gain > best_gain:
                best = feature
                best_gain = info_gain
        return best
                
    def _get_entropy(self, y):
        """
        Calculates the entropy of the binary distribution in a subset of target values.
        
        :arg y: Subset of target values.
        :type y: pandas Series.
        :return: Entropy of the binary distribution of y values.
        :return type: Float.
        """
        s_value = y.unique()[0]
        ps = y[y == s_value].count()
        if len(y.unique()) > 1:
            n_value = y.unique()[1]
            pn = y[y == n_value].count()
        else:
            pn = 0
        entropy = 0
        if ps != 0:
            entropy += -ps/(ps+pn)*math.log2(ps/(ps+pn))
        if pn != 0:
            entropy += -pn/(ps+pn)*math.log2(pn/(ps+pn))
        return entropy
            
    def print_tree(self):
        """
        Prints in text the decision tree model learned by the algorithm.
        """
        root = self.decision_tree
        if root is None:
            return
        print(root.value)
        if root.children is None:
            return
        for child in root.children:
            print(child, ":", root.children[child].value)
            self._recursive_print_tree(root.children[child], indent='\t')
            
    def _recursive_print_tree(self, node, indent):
        """
        Recursive method used for printing a subtree of the model.
        
        :arg node: Root node of the subtree to be printed.
        :type node: TreeNode.
        :arg indent: Indentation used to show hierarchy in the tree.
        :type indent: Char.
        """
        if node.children is None:
            return
        for child in node.children:
            print(indent, child, ":", node.children[child].value)
            self._recursive_print_tree(node.children[child], indent+'\t')

In [6]:
DT = DecisionTreeClassifier(max_depth=3)
DT.fit(X_train, y_train)

In [7]:
DT.print_tree()

physician-fee-freeze
y : synfuels-corporation-cutback
	 n : republican
	 y : republican
	 ? : republican
n : adoption-of-the-budget-resolution
	 y : democrat
	 n : democrat
	 ? : democrat
? : mx-missile
	 n : democrat
	 ? : republican
	 y : democrat


In [8]:
y_pred = DT.predict(X_test)
(y_pred == y_test.reset_index(drop=True)).value_counts()

True     80
False     3
dtype: int64

In [9]:
y_pred_train = DT.predict(X_train)
(y_pred_train == y_train.reset_index(drop=True)).value_counts()

True     318
False     14
dtype: int64